In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:

from autotst.species import Conformer

#from autotst.molecule import AutoTST_Molecule
#from ase.io import write
import ase
from ase.calculators.gaussian import *
#from autotst.calculators.gaussian import AutoTST_Gaussian
import subprocess
import shlex
from subprocess import call
from shlex import split
from ase.io.gaussian import read_gaussian_out
from rmgpy.molecule import Molecule as RMG_Molecule

In [ ]:
#subprocess.call(shlex.split('sbatch rotors_run_template.sh {0}'.format(geo_filename)))

In [21]:
from autotst.species import Species as TS_Species
from autotst.species import Conformer
from rmgpy.species import Species as RMG_Species
from rmgpy.molecule import Molecule as RMG_Molecule
from rdkit import Chem
import os
import arkane

class fake_statmech():
    
    def __init__(self):
        self.reactants = []
        self.products = []
        self.ts = None
        self.model_chemistry = 'M06-2X/cc-pVTZ'
        self.rotors = {}
        self.ark_gaus = None
        return
    
    
    def set_rotor_info(self, mol, torsion):
        
        a = min(torsion)
        b = max(torsion)
        tor_center = [a,b]
        
        tor_log = mol.toAugmentedInChIKey() + '_tor{0}{1}.log'.format(a,b)
        self.ark_gaus = arkane.gaussian.GaussianLog(tor_log)
        
        top_IDs = self.get_top_IDs(mol, tor_center)
        self.rotors[tuple(tor_center)] = "     HinderedRotor(scanLog=Log('{0}'), pivots={1}, top={2}, fit='fourier'),".format(tor_log, tor_center, top_IDs)
        return
    
    def get_top_IDs(self, mol, tor_center):
        assert len(tor_center) == 2
        assert isinstance(tor_center, list)
        tor_bond = None
        for bond in mol.getAllEdges():
            a = bond.atom1.id
            b = bond.atom2.id
            if (tor_center == [a,b]) or (tor_center == [b,a]):
                tor_bond = bond
                break
        assert tor_bond is not None

        mol.removeBond(tor_bond)
        tops = mol.split()
        assert len(tops) == 2
        
        return [atom.id for atom in tops[0].atoms]
    
    def get_all_ids(RMGBond, RMGMol, Found, Exclude):
        atom1 = bond.atom1
        atom2 = bond.atom2
        if (atom1.id not in Exclude) and (atom1.id not in Found):
            for bond in RMGMol.getAllBonds(atom1):
                Found += get_all_ids(bond, RMGMol, Exclude)
            Found.append(atom1.id)


        if (atom2.id not in Exclude) and (atom2.id not in Found):
            for bond in RMGMol.getAllBonds(atom2):
                Found += get_all_ids(bond, RMGMol, Exclude)
            Found.append(atom2.id)

        Found = sorted(set(Found))
        return Found
        
    def write_arkane_for_reacts_and_prods(self, species):
        """
        a method to write species to an arkane input file. Mol is an RMGMolecule
        """
        conf = Conformer(species.smiles[0])

        mol = conf.rmg_molecule
        freq_log = mol.toAugmentedInChIKey() + '_Freq'

        output = ['#!/usr/bin/env python',
                  '# -*- coding: utf-8 -*-', '', 'atoms = {']

        atom_dict = self.get_atoms(species)

        for atom, count in atom_dict.iteritems():
            output.append("    '{0}': {1},".format(atom, count))
        output = output + ['}', '']

        bond_dict = self.get_bonds(species)
        if bond_dict != {}:
            output.append('bonds = {')
            for bond_type, num in bond_dict.iteritems():
                output.append("    '{0}': {1},".format(bond_type, num))
            output.append("}")
        else:
            output.append('bonds = {}')

        label = Chem.rdinchi.InchiToInchiKey(
            Chem.MolToInchi(Chem.MolFromSmiles(mol.toSMILES()))).strip("-N")

        external_symmetry = mol.getSymmetryNumber()

        output += ["", "linear = False", "", "externalSymmetry = {}".format(external_symmetry), "",
                   "spinMultiplicity = {}".format(mol.multiplicity), "", "opticalIsomers = 1", ""]

        output += ["energy = {", "    '{0}': Log('{1}.log'),".format(
            self.model_chemistry, freq_log), "}", ""]

        output += ["geometry = Log('{0}.log')".format(freq_log), ""]

        output += ["frequencies = Log('{0}.log')".format(freq_log), ""]

        output += ["rotors = ["]
        for rotor, info in self.rotors.items():
            output += [info]
        output += ["]"]

        #input_string = ""

        #for t in output:
        #    #input_string += t + "\n"
        
        input_string = '\n'.join(output)

        with open(os.path.join(os.getcwd(), label +".py"), "w") as f:
            f.write(input_string)
        
        return input_string
            
    def get_atoms(self, species):
        """
        A method to create an atom dictionary for an rmg molecule
        """
        atom_dict = {}

        conf = Conformer(species.smiles[0])

        rmg_mol = conf.rmg_molecule

        for atom in rmg_mol.atoms:
            if atom.isCarbon():
                atom_type = "C"
            if atom.isHydrogen():
                atom_type = "H"
            if atom.isOxygen():
                atom_type = "O"

            try:
                atom_dict[atom_type] += 1
            except KeyError:
                atom_dict[atom_type] = 1

        return atom_dict

    def get_bonds(self, species):


        conf = Conformer(species.smiles[0])
        
        rmg_mol = conf.rmg_molecule

        bondList = []
        for atom in rmg_mol.atoms:
            for bond in atom.bonds.values():
                bondList.append(bond)
        bonds = list(set(bondList))
        bondDict = {}
        for bond in bonds:
            if bond.isSingle():
                if bond.atom1.symbol == 'C' and bond.atom2.symbol == 'C':
                    bondType = 'C-C'
                elif (bond.atom1.symbol == 'H' and bond.atom2.symbol == 'H'):
                    bondType = 'H-H'
                elif (bond.atom1.symbol == 'C' and bond.atom2.symbol == 'H') or (bond.atom1.symbol == 'H' and bond.atom2.symbol == 'C'):
                    bondType = 'C-H'
                elif (bond.atom1.symbol == 'O' and bond.atom2.symbol == 'O'):
                    bondType = 'O-O'
                elif (bond.atom1.symbol == 'C' and bond.atom2.symbol == 'O') or (bond.atom1.symbol == 'O' and bond.atom2.symbol == 'C'):
                    bondType = 'C-O'
                elif (bond.atom1.symbol == 'H' and bond.atom2.symbol == 'O') or (bond.atom1.symbol == 'O' and bond.atom2.symbol == 'H'):
                    bondType = 'O-H'
                elif bond.atom1.symbol == 'N' and bond.atom2.symbol == 'N':
                    bondType = 'N-N'
                elif (bond.atom1.symbol == 'C' and bond.atom2.symbol == 'N') or (bond.atom1.symbol == 'N' and bond.atom2.symbol == 'C'):
                    bondType = 'N-C'
                elif (bond.atom1.symbol == 'O' and bond.atom2.symbol == 'N') or (bond.atom1.symbol == 'N' and bond.atom2.symbol == 'O'):
                    bondType = 'N-O'
                elif (bond.atom1.symbol == 'H' and bond.atom2.symbol == 'N') or (bond.atom1.symbol == 'N' and bond.atom2.symbol == 'H'):
                    bondType = 'N-H'
                elif bond.atom1.symbol == 'S' and bond.atom2.symbol == 'S':
                    bondType = 'S-S'
                elif (bond.atom1.symbol == 'H' and bond.atom2.symbol == 'S') or (bond.atom1.symbol == 'S' and bond.atom2.symbol == 'H'):
                    bondType = 'S-H'
            elif bond.isDouble:
                if bond.atom1.symbol == 'C' and bond.atom2.symbol == 'C':
                    bondType = 'C=C'
                elif (bond.atom1.symbol == 'O' and bond.atom2.symbol == 'O'):
                    bondType = 'O=O'
                elif (bond.atom1.symbol == 'C' and bond.atom2.symbol == 'O') or (bond.atom1.symbol == 'O' and bond.atom2.symbol == 'C'):
                    bondType = 'C=O'
                elif bond.atom1.symbol == 'N' and bond.atom2.symbol == 'N':
                    bondType = 'N=N'
                elif (bond.atom1.symbol == 'C' and bond.atom2.symbol == 'N') or (bond.atom1.symbol == 'N' and bond.atom2.symbol == 'C'):
                    bondType = 'N=C'
                elif (bond.atom1.symbol == 'O' and bond.atom2.symbol == 'N') or (bond.atom1.symbol == 'N' and bond.atom2.symbol == 'O'):
                    bondType = 'N=O'
                elif (bond.atom1.symbol == 'O' and bond.atom2.symbol == 'S') or (bond.atom1.symbol == 'S' and bond.atom2.symbol == 'O'):
                    bondType = 'S=O'
            elif bond.isTriple:
                if bond.atom1.symbol == 'C' and bond.atom2.symbol == 'C':
                    bondType = 'C#C'
                elif bond.atom1.symbol == 'N' and bond.atom2.symbol == 'N':
                    bondType = 'N#N'
                elif (bond.atom1.symbol == 'C' and bond.atom2.symbol == 'N') or (bond.atom1.symbol == 'N' and bond.atom2.symbol == 'C'):
                    bondType = 'N#C'
            try:
                bondDict[bondType] += 1
            except KeyError:
                bondDict[bondType] = 1

        return bondDict

In [22]:
x = RMG_Molecule()
SML = 'CC(CO)(C)OC'
x.fromSMILES(SML)
piv_bond = None
i = 1
for atom in x.atoms:
    atom.id = i
    i+=1

for bond in x.getAllEdges():
    print bond.atom1.id, bond.atom2.id
    if (bond.atom1.id == 2 and bond.atom2.id == 4) or (bond.atom1.id == 2 and bond.atom2.id == 4):
        piv_bond = bond
print "Piv Bond: ", piv_bond

1 9
1 10
3 11
1 2
7 17
3 12
7 18
5 16
1 8
7 19
4 13
6 7
2 5
2 6
2 3
3 4
5 15
5 14
Piv Bond:  None


In [23]:
from autotst.species import Species as TS_Species

SML = 'CC(CO)(C)OC'
aspec = TS_Species(smiles=[SML])
xmol = aspec.conformers.values()[0][0].rmg_molecule
#print xmol.toAugmentedInChIKey()
i = 1
for atom in xmol.atoms:
    atom.id = i
    i += 1

for bond in xmol.getAllEdges():
    x = bond.atom1
    y = bond.atom2
    
    if x.element.symbol == 'C' and y.element.symbol == 'C':
        #print x.element, x.id, y.id, y.element
        #print
        break
        
b = fake_statmech()
tor = [bond.atom1.id, bond.atom2.id]
print tor
b.set_rotor_info(xmol, tor)
b.write_arkane_for_reacts_and_prods(aspec)

thermoClass = 'NASA'
rmg_spec = RMG_Species()
rmg_spec.fromSMILES(SML)
#ark_spec = arkane.ArkaneSpecies(species=rmg_spec)
ark_therm = arkane.ThermoJob(rmg_spec, thermoClass)
ark_therm.arkane_species.use_hindered_rotors = True

[3, 6]


In [20]:
modelChemistry = 'M06-2X/cc-pVTZ'
species = 'my_fav_species'
spec_file = 'VMPUAIZSESMILD-UHFFFAOYSA.py'
output = ['#!/usr/bin/env python',
          '# -*- coding: utf-8 -*-',
          '',
          'modelChemistry = "{0}"'.format(modelChemistry),
          'useHinderedRotors = True',
          'useBondCorrections = True',
          '',
          "species('{0}', '{1}')\n\nstatmech('{0}')".format(species, spec_file),
          "thermo('{0}', '{1}')".format(species, 'NASA')]


output = '\n'.join(output)
print output
with open('thermo_input.py', 'w') as f:
    f.write(output)

#!/usr/bin/env python
# -*- coding: utf-8 -*-

modelChemistry = "M06-2X/cc-pVTZ"
useHinderedRotors = True
useBondCorrections = True

species('my_fav_species', 'VMPUAIZSESMILD-UHFFFAOYSA.py')

statmech('my_fav_species')
thermo('my_fav_species', 'NASA')


In [21]:
input_file = 'thermo_input.py'
out_path = os.getcwd()
ark = arkane.Arkane(inputFile=input_file, outputDirectory=out_path)
print ark.inputFile
ark.plot = False
ark.execute()

thermo_input.py
Arkane execution initiated at Wed Feb 20 11:04:22 2019

################################################################
#                                                              #
# Automated Reaction Kinetics and Network Exploration (Arkane) #
#                                                              #
#   Version: 2.3.0                                             #
#   Authors: RMG Developers (rmg_dev@mit.edu)                  #
#   P.I.s:   William H. Green (whgreen@mit.edu)                #
#            Richard H. West (r.west@neu.edu)                  #
#   Website: http://reactionmechanismgenerator.github.io/      #
#                                                              #
################################################################

Loading species my_fav_species...
Assigned a frequency scale factor of 0.955 for model chemistry M06-2X/cc-pVTZ


Loading statistical mechanics parameters for my_fav_species...
Determined a symmetry number of 3 f

In [30]:
thermo = ark.speciesDict["my_fav_species"].thermo
thermo

NASA(polynomials=[NASAPolynomial(coeffs=[3.44017,0.0475804,3.77931e-05,-1.17407e-07,7.86823e-11,-51944.2,8.02123], Tmin=(10,'K'), Tmax=(402.565,'K')), NASAPolynomial(coeffs=[1.34124,0.0684361,-3.9918e-05,1.12868e-08,-1.23952e-12,-51775.2,16.2375], Tmin=(402.565,'K'), Tmax=(3000,'K'))], Tmin=(10,'K'), Tmax=(3000,'K'), E0=(-431.949,'kJ/mol'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(453.139,'J/(mol*K)'))

In [34]:
ark_info = arkane.gaussian.GaussianLog('VMPUAIZSESMILD-UHFFFAOYSA-N_tor17.log')
ark_info.loadScanEnergies

<bound method GaussianLog.loadScanEnergies of <arkane.gaussian.GaussianLog instance at 0x2b90a4ba9098>>

In [ ]:
import arkane
import cclib

from ase.calculators.gaussian import *
from ase.io.gaussian import read_gaussian_out

from rmgpy.molecule import Molecule as RMG_Molecule

from autotst.species import Species as TS_Species
from autotst.species import Conformer

import matplotlib.pyplot as plt
import numpy as np

class hindered_rotors:
    
    def __init__(self, SMILES, Label=None, directory=None):
        if isinstance(SMILES, str):
            self.SMILES = SMILES
            
            self.path = directory
            
            self.__RMGMol__ = None
            
            self.geoIsOpt = False
            
            tmpMol = RMG_Molecule()
            tmpMol.fromSMILES(SMILES)
            self.AugInchiKey = tmpMol.toAugmentedInChIKey()
            self.label = self.AugInchiKey.strip('-N')
            
            #Atom indices must start from 1!!! (As Gaussain does but not others)
            #TODO fix issue with matching torsion to atom labels
            self.torsions = None
            
            self.scans = []

        return
    
    def write_Geo_Input(self):
        # Preparing gaussian input file for Geo Opt
        gaus_job = Gaussian()
        gaus_job.label = self.label + '_Geo'
        gaus_job.parameters['method'] = 'm062x'
        gaus_job.parameters['basis'] = '6-311+g(2df,2p)'
        gaus_job.extra = 'opt=(calcfc,maxcycle=1000)'
        del gaus_job.parameters['force']

        # Write geo input file
        gaus_job.write_input(.ase_molecule)
    
    def write_Freq_Input(self, freq_file=None,  path=None, options=None):
        # Preparing gaussian input file for Freq Calc
        if self.geoIsOpt:
            mol = self.getRMGMol()
            mol.updateMultiplicity()
            
            if freq_file is None:
                freq_file = self.label + '_Freq.com'
                
            if path is None:
                path = self.path
            
            if options is None:
                options = ['%nprocshared=20',
                           '%mem=5GB',
                           '#p m062x/6-311+g(2df,2p) freq iop(7/33=1)']

            output = '\n'.join(options)
            
            output += '\n'.join(['','','0 {}'.format(mol.multiplicity),''])

            n = 1
            for atom in mol.atoms:
                assert n == atom.id
                n += 1
                output += "{}     {}     {}     {}\n".format(atom.element, atom.coords[0], atom.coords[1], atom.coords[2])
            
            with open(os.path.join(path, freq_file), 'w') as f:
                f.write(output)
                f.close()
            return
    
    def getRMGMol(self):
        return self.__RMGMol__
            
    def updateMol(self, geo_file=None, path=None):
        assert self.__RMGMol__ is None, "Highly recommended not to update molecule more than once"
        
        if geo_file is None:
            geo_file = self.label + '_Geo.log'
        if path is None:
            path = self.path
        
        atoms = None
        try:
            atoms = read_gaussian_out(geo_file)
        except:
            print "Failed to Find Valid Geo_file"
            return False
        
        #Update XYZ
        mol = RMG_Molecule()
        mol.fromXYZ(atoms.get_atomic_numbers(), atoms.get_positions())
        
        #Update IDs
        i = 1
        for atom in mol.atoms:
            atom.id = i
            i += 1
        
        #Update Multiplicity
        mol.updateMultiplicity()
        
        self.__RMGMol__ = mol
        self.geoIsOpt = True
        return True
            
    def set_torsions(self):
        """
        Method setting torsions of simple chains to hindered rotors object

        NO RINGS
        NO RINGS
        NO RINGS

        A - Bond1 - B - Bond2 - C - Bond3 - D

        i = A.index
        j = B.index
        k = C.index
        l = D.index

        Torsion is unique if j,k & k,j pairing is unique

        """

        if not self.geoIsOpt:
            self.updateMol()
        
        assert self.geoIsOpt
        mol = self.getRMGMol()
 
        # dict of (j, k) key with [i,j,k,l] value
        # Wipes existing torsions away to recalculate
        self.torsions = {}
            
        for bond2 in mol.getAllEdges():
            i, j, k, l = -1,-1,-1,-1
            B = bond2.atom1
            j = bond2.atom1.id

            C = bond2.atom2
            k = bond2.atom2.id

            assert (j,k) not in self.torsions.keys() or (k,j) in torsions.keys()
            # [i,j,k,l] and [i,j,k,m] torsion is the same to Gaussian as long as center is k&j or j&k
            #continue

            found_bond1 = False
            found_bond3 = False

            for Bbond in mol.getBonds(B).values():
                if (Bbond.atom1.id == B.id) and (Bbond.atom2.id != C.id):
                    #Bbond is not bond2
                    #Bbond must be bond1
                    found_bond1 = True
                    i = Bbond.atom2.id
                elif (Bbond.atom1.id != C.id) and (Bbond.atom2.id == B.id):
                    #Bbond is not bond2
                    #Bbond must be bond1
                    found_bond1 = True
                    i = Bbond.atom1.id
                else:
                    #Bbond is bond2
                    assert (Bbond.atom1 == C) or (Bbond.atom2 == C)

            for Cbond in mol.getBonds(C).values():
                if (Cbond.atom1.id == C.id) and (Cbond.atom2.id != B.id):
                    #Bbond is not bond2
                    #Bbond must be bond3
                    found_bond3 = True
                    l = Cbond.atom2.id
                elif (Cbond.atom1.id != B.id) and (Cbond.atom2.id == C.id):
                    #Bbond is not bond2
                    #Bbond must be bond1
                    found_bond3 = True
                    l = Cbond.atom1.id
                else:
                    #Cbond is bond2
                    assert (Cbond.atom1 == B) or (Cbond.atom2 == B)

            #print found_bond1, found_bond3
            #print i, j, k, l
            #print
            if found_bond1 and found_bond3:
                self.torsions[(j,k)] = [i,j,k,l]
        return
    
    def create_scan(self, torsion, steps=None, stepsize=None, path=None, label=None):
        """
        Creates scan object and adds it to rotors class
        
        tor_center          :: list of atom ids (relative to self.__RMGMol__) of central atoms of dihedral angle
        steps               :: number of steps taken during scan
        stepsize            :: degrees between steps
        path                :: general path where things like geometery logs, scan com, and scan log can be found
        """
        assert len(tor_center) == 4
        assert isinstance(steps, int)
        assert isinstance(stepsize, float)
        
        if self.geoIsOpt:
            mol = self.getRMGMol()
            
            scan_inst = scan(mol, torsion, steps, stepsize, path=path, label=None)
            scans.append(scan_inst)
        return
    
    
    def generate_all_scans(self, steps=None, stepsize=None,path=None, label=None):
        assert self.torsions is not None
        
        for center, torsion in self.torsions.items():
            self.create_scan(torsion, steps=steps, stepsize=stepsize, path=path, label=lable)
        return
    
    def update_scans(self):
        updated = True
        
        for scan in self.scans:
            got_data = scan.set_data()
            updated = updated and got_data
        
        return self.scansUpdated
    
    def Rote_write_geo_com(self, filename=None):
        """
        Writes Gaussian input file for a geometry optimization of self.__RMGMol__
        with a basisand parameters similar to AutoTST
        """
        auto_mol = None
        if filename is None:
            filename = self.AugInchiKey + '_Geo.com'

        # Preparing gaussian input file parameters for Geo Opt as AutoTST does it
        g_inst = Gaussian()
        g_inst.label = self.AugInchiKey + '_Geo'
        
        g_inst.parameters['method'] = 'm062x'
        g_inst.parameters['basis'] = '6-311+g(2df,2p)'
        g_inst.extra = 'opt=(calcfc,maxcycle=1000)'
        del g_inst.parameters['force']

        # Write Gaussian input file
        # TODO Fix
        test_g.write_input(auto_mol.ase_molecule)
        return False
    
    def Create_Stat_Mech():
        return
        

In [ ]:
class scan:
    
    def __init__(self, RMGMol, torsion, steps=None, stepsize_deg=None, path=None, label=None):
        
        self.__RMGMol__ = RMGMol
        self.AugInchiKey = RMGMol.toAugmentedInChIKey()
        
        if label is None
            label = self.AugInchiKey.strip('-N')
        
        self.path = path
        self.label = label
        
        self.i = torsion[0]
        self.j = torsion[1]
        self.k = torsion[2]
        self.l = torsion[3]
        
        self.steps = steps
        self.stepsize_deg = stepsize_deg
        self.stepsize_rad = None
        
        self.geo_log = None
        self.input_com = None
        self.output_log = None
        
        #After running a scan, populate data
        self.data = None
        
        self.opt_indices = None
        self.start_indices = None
        self.opt_scfenergies = None
        self.atomcoords = None
        
        
        if defaults:
            self.set_default_files()

    def set_default_files(self):
        self.geo_log = self.label + '_Geo.log'

        a = min([self.j, self.k])
        b = max([self.j, self.k])
        
        self.input_com = self.label + '_tor{}{}'.format(a, b) + '.com'
        self.output_log = self.label + '_tor{}{}'.format(a, b) + '.log'
        return
    
    def write_Tor_input(self, filename=None, path=None, options=None):
        """
        Given filename and options, and taking RMG_Molecule object from scan
        Write input file for torsion of scan
        """
        if filename is None:
            filename = self.input_com
        
        if path is None:
            path = self.path
        
        torsion = self.torsion
        
        mol = self.__RMGMol__
        mol.updateMultiplicity()
        
        if options is None:
            options = ['%nprocshared=20',
                       '%mem=5GB',
                       '#p m062x/6-311+g(2df,2p) Opt=(CalcFC,ModRedun)']
        
        output = '\n'.join(options)
        output += '\n'.join(['','','0 {}'.format(mol.multiplicity),''])
        
        n = 1
        for atom in mol.atoms:
            assert n == atom.id
            n = n + 1
            output = output + "{}     {}     {}     {}\n".format(atom.element, atom.coords[0], atom.coords[1], atom.coords[2])

        output = output + '\n'

        for bond in mol.getAllEdges():
            output = output + 'B {0} {1}\n'.format(bond.atom1.id, bond.atom2.id)

        output = output + 'D {0} {1} {2} {3} S {4} {5}'.format(torID[0],
                                                               torID[1],
                                                               torID[2],
                                                               torID[3],
                                                               nsteps,
                                                               stepsize)
        output = output + '\n\n\n'
        
        a = min(torsion[1:3])
        b = max(torsion[1:3])
        tor_file = self.AugInchiKey + '_tor{0}{1}'.format(a,b) + '.com'
        
        with open(tor_file, 'w') as F
            #F.write('')
            F.write(output)
            F.close
        return
    
    def set_data_Arkane(self, scan_log=None):
        if scan_log is None:
            scan_log = self.output_log
        
        ark = arkane.gaussian.GaussianLog(scan_log)
        
        scan_energy = ark.loadScanEnergies()
        
        self.ark_energies = scan_energy[0]
        self.ark_thetas = scan_energy[1]
        
        return 
        
    def set_data(self, scan_log=None):
        #Populating scan data attributes from gaussian scan output log
        if scan_log is None:
            scan_log = self.output_log
        
        try:
            self.data = cclib.io.ccread(data_log)
            ark = arkane.gaussian.GaussianLog(scan_log)
        except:
            print "Failed to read data log. Could be file does not exist or not in proper directory"
            return False
        scan_energy = ark.loadScanEnergies()
        
        self.ark_energies = scan_energy[0]
        self.ark_thetas = scan_energy[1]
        
        assert len(self.ark_thetas) == len(self.ark_energies)
        
        self.opt_indices = [i for i, status in enumerate(self.data.optstatus) if status==2]
        self.start_indices = [i for i, status in enumerate(self.data.optstatus) if status==1]
        self.opt_scfenergies = [self.data.scfenergies[index] for index in self.opt_indices]
        
        assert len(self.opt_scfenergies) == len(self.ark_energies)
        self.steps = len(self.opt_scfenergies) - 1
        
        self.stepsize_rad = self.ark_thetas[1] - self.ark_thetas[0]
        assert self.stepsize/360 - self.stepsize_rad/(2*3.1415) < 0.01
        assert len(self.opt_indices) == self.steps + 1
        assert len(self.start_indices) == self.steps + 1

        
        #Keeping atomcoords organized with their respective atom nos for easy ID
        atom_id_coords = []
        atomids = self.data.atomnos
        for i, geo_coords in enumerate(self.data.atomcoords):
            #Setting up [atomno, x, y, x] for each atom in a geometry
            geo_id_coords = np.insert(geo_coords, 0, atomids, axis=1)
            #Adding all of individual geometries to master list
            atom_id_coords.append(geo_id_coords)
        
        self.atomcoords = atom_id_coords
        
        return True
    
    def plot_scan(self):
        plt.plot(self.ark_thetas, self.ark_energies)
        return
    
    
    def check_theta_continuous(self, tol=None):
        """
        Returns true if scfenergies at the same theta are within tolerance
        
        Must populate:
        self.opt_scfenergies
        self.steps
        self.stepsize
        
        """
        if tol is None:
            tol = 10**-8
        
        energy_by_theta = {}
        
        theta = 0
        for energy in self.opt_scfenergies:
            #Assumes stepsize given in degrees
            #Assume thetas varying by more than a degree are different (see slope_continous)
            
            if theta in energy_by_theta.keys():
                previous = energy_by_theta[theta]
                error = 1.00000000-previous/energy
                
                if (error > tol) or (error < -tol):
                    return False
            else:
                energy_by_theta[theta] = energy
            
        return True
    
    
    def check_slope_continuous(self, tol=None):
        """
        Check if change in energy within given tolerance over opt scf energies
        
        Must populate:
        self.opt_scfenergies
        self.steps
        self.stepsize
        
        """
        if tol is None:
            tol = 10**-2.5
        
        for i in range(1, len(self.opt_scfenergies)):
            slope = (self.opt_scfenergies[i]-self.opt_scfenergies[i-1])/self.stepsize_deg
            print slope
            if slope>tol:
                return False
        
        return True
    
    
    def check_min(self):
        min_en = self.ark_energies[0]
        for energy in self.ark_energies:
            if energy < min_en
                return False
        
        min_en = self.opt_scfenergies[0]
        for energy in self.opt_scfenergies:
            if energy < min_en
                return False
        
        return True

In [90]:
#hin_inst = hindered_rotors('CCCC(=O)OC')
hin_inst = hindered_rotors('CC(CO)(C)OC', default=False)
hin_inst.updateMol()
hin_inst.set_torsions()
hin_inst.set_scans(36, 10.0)

for tor_scan in hin_inst.scans:
    tor_scan.get_data()
    print tor_scan.j, tor_scan.k
    print tor_scan.geo_log
    print tor_scan.input_com
    print tor_scan.output_log
    break

7 1
VMPUAIZSESMILD-UHFFFAOYSA-N_Geo.log
VMPUAIZSESMILD-UHFFFAOYSA-N_tor17.com
VMPUAIZSESMILD-UHFFFAOYSA-N_tor17.log


In [91]:
hin_inst.updateMol()
freq_file =hin_inst.write_Freq_Input()

In [60]:
x = Conformer('CC(CO)(C)OC')

In [ ]:
data=tor_scan.data
steps = 36
theta = 10.0

opt_indices = [i for i, status in enumerate(data.optstatus) if status==2]
opt_energies = [data.scfenergies[index] for index in opt_indices]
len(opt_energies)

thetas = [i*theta for i in range(steps+1)]
plt.plot(thetas, opt_energies)
#plt.plot(opt_energies)
plt.show()

In [93]:
print freq_file
base_file = freq_file[:-9]
print base_file

VMPUAIZSESMILD-UHFFFAOYSA-N_Freq.com
VMPUAIZSESMILD-UHFFFAOYSA-N


In [94]:
import subprocess, shlex
base_filename = 'VMPUAIZSESMILD-UHFFFAOYSA-N_Freq'
subprocess.call(shlex.split('sbatch rotors_run_template.sh {0}'.format(base_filename)))

0

In [ ]:
# To create new template run-script

#rotors_runscript_template = "#!/bin/bash\n\n#SBATCH --job-name=$1\n#SBATCH --output=$1.log\n\n## number of nodes\n#SBATCH -N 1\n#SBATCH --exclusive\n#SBATCH --partition=general\n#SBATCH --mem=120000\n\n## set the gaussian scratch directory to a fast drive\n## note that /tmp/ may be even faster than /gss_gpfs_scratch/\n#export GAUSS_SCRDIR=/scratch/$USER/gaussian_scratch\n## make the directory if it doesn't exist already\n#mkdir -p $GAUSS_SCRDIR\n\n# run gaussian, with the desired input file\ng16 $1.com\n\n"

lst_template = ["#!/bin/bash",
                "",
                "#SBATCH --job-name=rotor",
                "#SBATCH --output=$1.log",
                "",
                "## number of nodes",
                "#SBATCH -N 1",
                "#SBATCH --exclusive",
                "#SBATCH --partition=general",
                "#SBATCH --mem=120000",
                "",
                "## export GAUSS_SCRDIR=/scratch/$USER/gaussian_scratch",
                "## make the directory if it doesn't exist already",
                "## mkdir -p $GAUSS_SCRDIR",
                "",
                "# run gaussian, with the desired input file",
                "g16 $1.com",
                "",
                ""]

#new_script_template = open('rotors_run_template.sh', 'w')
#new_script_template.write('\n'.join(lst_template))
print '\n'.join(lst_template)
#new_script_template.close()
#print rotors_runscript_template


In [ ]:
# Writing Geo opt input as AutoTST would

from autotst.species import Conformer
from ase.calculators.gaussian import *
test_mol = Conformer('CCCC(=O)OC')

# Preparing gaussian input file parameters for Geo Opt as AutoTST does it
test_g = Gaussian()
test_g.label = test_mol.rmg_molecule.toAugmentedInChIKey() + '_Geo'
print test_g.label + '.com'
print
test_g.parameters['method'] = 'm062x'
test_g.parameters['basis'] = '6-311+g(2df,2p)'
test_g.extra = 'opt=(calcfc,maxcycle=1000)'
del test_g.parameters['force']

# Checking parameters are good
for key, val in test_g.parameters.items():
    print key, val

# Write geo input file
#test_g.write_input(test_mol.ase_molecule)
test_mol.view()

In [41]:
# Writing Freq input as AutoTST would

from autotst.species import Conformer
from ase.calculators.gaussian import *
test_mol = Conformer('CC(CO)(C)OC')

# Preparing gaussian input file parameters for Geo Opt as AutoTST does it
test_gf = Gaussian()
test_gf.label = test_mol.rmg_molecule.toAugmentedInChIKey() + '_Freq'
print test_gf.label + '.com'
print
test_gf.parameters['method'] = 'm062x'
test_gf.parameters['basis'] = '6-311+g(2df,2p)'
test_gf.extra = 'freq iop(7/33=1)'
del test_gf.parameters['force']
test_gf.atoms

# Checking parameters are good
for key, val in test_gf.parameters.items():
    print key, val

# Write geo input file
#test_g.write_input(test_mol.ase_molecule)
#test_mol.view()

VMPUAIZSESMILD-UHFFFAOYSA-N_Freq.com

charge 0
method m062x
basis 6-311+g(2df,2p)


In [ ]:
#!/bin/bash

#SBATCH --job-name={}
#SBATCH --output={}.log

## number of nodes
#SBATCH -N 1
#SBATCH --exclusive
#SBATCH --partition=general
#SBATCH --mem=120000

## set the gaussian scratch directory to a fast drive
## note that /tmp/ may be even faster than /gss_gpfs_scratch/
#export GAUSS_SCRDIR=/gss_gpfs_scratch/$USER/gaussian_scratch
## make the directory if it doesn't exist already
#mkdir -p $GAUSS_SCRDIR

# run gaussian, with the desired input file
g16 {}.com